hier einmal die hoffentlich schöne Version des Notebooks, welches es erlaubt alle Anläufe von Kreuzfahrtschiffen in Kiel herunterzuladen. 
Quelle:https://kreuzfahrtschiffehamburg.de/hafen-kiel/

In [82]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [83]:
#define function to get the data from the website
def get_data(year):
    url = f'https://kreuzfahrtschiffehamburg.de/kiel-{year}/'
    response = requests.get(url)
    html_content = response.text
    #parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    #text = soup.get_text()
    # Extract text from all paragraphs
    paragraphs = soup.find_all('p')
    paragraphs = [p.get_text() for p in paragraphs]
    #print(paragraphs)


    #I want to remove \xa0 \xa from the text

    removed =[]
    for p in paragraphs:
        text = p.replace('\xa0', ' ')
        #text = text.strip()
        #text = text.split('\n')
        removed.append(text)

    #print(removed)

    #remove first 7 elements
    removed = removed[2:]
    removed = removed[:-4]
    #print(removed)

    return removed

In [84]:
#define function to clean the data
def split_list_by_word(input_list, split_word):
    """
    Splits a list into sublists each time the split_word is encountered.

    Parameters:
    input_list (list): The list to be split.
    split_word (str): The word to split the list by.

    Returns:
    list: A list of sublists.
    """
    sublists = []
    temp_list = []

    for entry in input_list:
        if split_word in entry:
            if temp_list:
                sublists.append(temp_list)
                temp_list = []
        temp_list.append(entry)

    if temp_list:
        sublists.append(temp_list)

    
    print(sublists)

    return sublists



In [85]:
#delete the first 5 elements of each sublist
def remove_first_entries(sublists):
    sublists_cleaned = []
    for list in sublists:
        list = list[5:]
        sublists_cleaned.append(list)

    print(sublists_cleaned)
    return sublists_cleaned

In [86]:
#define function to get numbers of entries in each sublist
def get_numbers(sublists_cleaned):
    rows = []
    for sublist in sublists_cleaned:
        #get the indices of the days of the week
        indices = [i for i, x in enumerate(sublist) if x in ['Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'So']]
        if not indices:
            continue  # skip sublist if no days of the week are found
        #print(indices)
        #add up the indices to get the number of rows in each dataframe
        numbers = len(indices)
        rows.append(numbers)
        
    print(rows)
    return rows
    #print(rows)

In [87]:
#define function to create the df and concatenate the data
def create_df(sublists, rows):
    dataframes = []
    for list,number in zip(sublists, rows):
        print(list)
        print(number)
        #split list in chunks
        #chunks = [list[i:i + number] for i in range(0, len(list), number)]
        #df = pd.DataFrame(chunks).transpose()
        # Rename columns for clarity
        #df.columns = ['Weekday','Arrival_date', 'Departure_date','Ship','Pier']
        #dataframes.append(df)
'''
    # Concatenate the dataframes
    df_complete = pd.concat(dataframes, ignore_index=True)

    #reshape the arrival and departure dates
    df_final = df_complete
    print(type(df_complete['Arrival_date']))
    df_final['Arrival_date'] = df_complete['Arrival_date'].str.split(' ').str[0]
    df_final['Departure_date'] = df_complete['Departure_date'].str.split(' ').str[0]
    print(df_final)
    return df_complete
'''
   

"\n    # Concatenate the dataframes\n    df_complete = pd.concat(dataframes, ignore_index=True)\n\n    #reshape the arrival and departure dates\n    df_final = df_complete\n    print(type(df_complete['Arrival_date']))\n    df_final['Arrival_date'] = df_complete['Arrival_date'].str.split(' ').str[0]\n    df_final['Departure_date'] = df_complete['Departure_date'].str.split(' ').str[0]\n    print(df_final)\n    return df_complete\n"

In [88]:
#define function to save the df to a csv file
def save_df(df, year):
    df.to_csv(f'kiel_{year}.csv', index=False)

Nun habe ich alle funktionen erstellt. Es sollte nun möglich sein für jedes Jahr die Daten runter zu laden und als csv Dateien abzuspeichern

In [89]:
#lets try and get the data for 2012
year = 2012
removed = get_data(year)
sublists = split_list_by_word(removed, 'Tag')
sublists_cleaned = remove_first_entries(sublists)
rows = get_numbers(sublists_cleaned)
#df = create_df(sublists, rows)
#save_df(df, year)




[['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'So', '31.03.12  08:00', '31.03.12  17:00', 'AIDAcara', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'So', 'So', 'So', 'So', 'So', '07.04.12  08:00', '14.04.12  08:00', '21.04.12  08:00', '28.04.12  08:00', '28.04.12  08:00', '07.04.12  17:00', '14.04.12  17:00', '21.04.12  17:00', '28.04.12  17:00', '28.04.12  16:00', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'AIDAcara', 'MSC Lirica', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai', 'Ostseekai'], ['Tag', 'Ankunft   Uhrzeit', 'Abfahrt   Uhrzeit', 'Schiff', 'Liegeplatz', 'Mi', 'Do', 'Sa', 'Sa', 'So', 'Mo', 'Mo', 'Do', 'Sa', 'Sa', 'Mo', 'Di', 'Di', 'Do', 'Sa', 'Sa', 'Sa', 'So', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'Sa', 'Sa', 'Mi', '02.05.12  08:00', '03.05.12  09:00', '05.05.12  08:00', '05.05.12  10:00', '06.05.12  09:00', '07.05.12  07:00', '07.05.12  09:00', '10.05.12  10:00', '12.05.12  08:00', '12.05.12  10:00', '14.05.12  08